<a href="https://colab.research.google.com/github/vlozg/speech_hmm/blob/main/Test_DiagHMM_009.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LƯU Ý: NOTEBOOK NÀY CHỈ DÙNG ĐỂ SHOW KẾT QUẢ CHẠY, KHÔNG NÊN CHẠY LẠI NOTEBOOK NÀY 
(VÌ TRONG NÀY CÓ CODE LƯU LẠI PRETRAINED MODEL LÊN DRIVE SẼ BỊ XÓA)

# Speech to text with HMM

- **Bài toán**: Chuyển giọng nói thành văn bản
    - **Input**: Đoạn ghi âm chứa nội dung là các số từ 0 đến 9
    - **Output**: Phân lớp của đoạn ghi âm

# Các biến thiết lập cho thử nghiệm

In [ ]:
n_mfcc_ceptrum = 12
n_delta_features = 0
fsdd_split = 0.6
wolfram_split = 0.6
experiment_id = '009'

In [ ]:
n_mfcc_features = n_mfcc_ceptrum * (1+n_delta_features)
n_mfcc_features

12

# Import và cài đặt thư viện

In [ ]:
# cài lib. note: cài xong phải restart runtime
!pip install pydub
!pip install pomegranate

In [ ]:
# Xác thực google để upload/download qua google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Quản lý file, folder
import os
from shutil import copyfile, rmtree
import random

# Xử lý audio
import librosa
import librosa.display
from scipy.io import wavfile

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pomegranate # Thư viện cho mô hình xác suất
from pomegranate import *

# Tải dữ liệu và lấy xác thực Google

Dữ liệu dùng để huấn luyện và đánh giá, còn xác thực google thì dùng để upload/download mô hình trên drive.

In [ ]:
%%capture
# download wolfram
if not os.path.isfile('./dataset_1_wolfram.zip'):
  !gdown --id 115tIAitBNeJC0DwrP-ZyJ6RS3TyWN0qD
  !unzip -o dataset_1_wolfram.zip

# dowload FSDD
if not os.path.isfile('./dataset_2_FSDD.zip'):
  !gdown --id 1Ua9zlPBc0Fv4xGHSQTb7eIvUh_dqFI6P
  !unzip -o dataset_2_FSDD.zip

# download self recorded audio
!gdown --id 1lH_k1AYMVlJvodtZdD7OK2zkdPXxlW9i

In [ ]:
# Lấy xác thực google để upload/download file
auth.authenticate_user()
gauth =  GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Hàm xử lý âm thanh

In [ ]:
def scaleAddNoise(wave):
  return ((wave - wave.mean()) / wave.std()) + 0.2*np.random.normal(size=wave.shape)

def read_process_audio(full_audio_path):
  # Bắt buộc âm thanh đọc vào phải cùng sample rate
  sample_rate, wave =  wavfile.read(full_audio_path)
  if sample_rate != 16000:
    wave, sample_rate =  librosa.load(full_audio_path, sr=16000) # Hàm đọc của librosa quá chậm, scipy nhanh gấp 5-10 lần
  else:
    wave=wave/32768 # Chuẩn hóa về số thực
  wave = np.pad(wave, (2000,2000), 'constant', constant_values=(0.0,0.0))
  wave = scaleAddNoise(wave)
  return wave, sample_rate

def extract_mfcc(wave, sample_rate):
  mfccs = librosa.feature.mfcc(y=wave, sr=sample_rate, n_mfcc=n_mfcc_ceptrum+2, n_mels=40, lifter=40)[2:,:].T/800
  return mfccs

def mfcc_delta_features(mfcc, order):
  if order==0:
    return mfcc
  dmfcc = librosa.feature.delta(mfcc, order=order)
  return dmfcc

def full_mfcc_from_file(full_audio_path):
  wave, sr = read_process_audio(full_audio_path)
  mfccs = extract_mfcc(wave, sr)
  full_mfccs = mfccs
  if n_delta_features >= 1:
    mfccs_d1 = mfcc_delta_features(mfccs, 1)
    full_mfccs = np.hstack([full_mfccs,mfccs_d1])
  if n_delta_features >= 2:
    mfccs_d2 = mfcc_delta_features(mfccs, 2)
    full_mfccs = np.hstack([full_mfccs,mfccs_d2])
  return full_mfccs

# Hàm bổ trợ

In [ ]:
def buildDataSet(dir):
    # Filter out the wav audio files under the dir
    fileList = [f for f in os.listdir(dir) if os.path.splitext(f)[1] == '.wav']
    dataset = {}
    for fileName in fileList:
        tmp = fileName.split('.')[0]
        label = tmp.split('_')[1]

        # label = filename.split('_')[0]
        feature = full_mfcc_from_file(dir+fileName)
        if label not in dataset.keys():
            dataset[label] = []
            dataset[label].append(feature)
        else:
            exist_feature = dataset[label]
            exist_feature.append(feature)
            dataset[label] = exist_feature
    return dataset

def makeTrainTestDir(mainFolder, filenames, rate):
    paths = [f'{mainFolder}',
             f'{mainFolder}/train_audio',
             f'{mainFolder}/test_audio']

    for path in paths:
        try:
            os.mkdir(path)
        except:
            rmtree(path)
            os.mkdir(path)

    random.seed(1)
    random.shuffle(filenames)
    splitPoint = int(len(filenames)*rate)
    trainFilenames = filenames[:splitPoint]
    testFilenames = filenames[splitPoint:]

    for filename in trainFilenames:
        copyfile(filename[0], f"{paths[1]}/{filename[1]}")

    for filename in testFilenames:
        copyfile(filename[0], f"{paths[2]}/{filename[1]}")

# Setup thư mục chứa data

In [ ]:
def formatFilenameFSDD(dir):
    filenames = []
    count = 0
    for filename in os.listdir(dir):
        tmp = str(count) + '_' + filename.split('_')[0] + '.wav'
        filenames.append((f"{dir}/{filename}", tmp))
        count += 1
    return filenames

filenames = formatFilenameFSDD('./dataset_2_FSDD')
makeTrainTestDir('fsdd', filenames, fsdd_split)

In [ ]:
def formatFilenameWolfram(dir):
    filenames = []
    count = 0
    folders = os.listdir(dir)
    for folder in folders:
        for filename in os.listdir(f'{dir}/{folder}'):
            tmp = str(count) + '_' + folder.split('_')[0] + '.wav'
            filenames.append((f"{dir}/{folder}/{filename}", tmp))
            count += 1

    return filenames

filenames = formatFilenameWolfram('./dataset_1_wolfram')
makeTrainTestDir('wolfram', filenames, wolfram_split)

# Huấn luyện mô hình (có thể bỏ qua vì mô hình đã save trên drive)

## Hàm train mô hình

In [ ]:
def Generate_DiagGMM(full_fset, n_features, n_states=5, n_cmps=3):
  means = np.mean(full_fset, axis=0)
  stds = np.std(full_fset, axis=0)
  # initial values for all gaussian components
  np.random.seed(None)
  dist_init = np.random.random((n_states, n_cmps, n_features, 2))
  dist_init[..., 0] -= 0.5  # center means to 0.0
  #for feat_i in range(n_features):
      # random init mean in range [-std, std)
      #dist_init[..., feat_i, 0] *= 2 * stds[feat_i]
      #dist_init[..., feat_i, 0] += means[feat_i]
      # random init std in range 1std/n_components
      #dist_init[..., feat_i, 1] *= stds[feat_i] / n_cmps

  dists = tuple(
      GeneralMixtureModel(list(
          IndependentComponentsDistribution(tuple(
              NormalDistribution(*dist_init[state_i, cmp_i, feat_i, :])
              for feat_i in range(n_features)
          ))
          for cmp_i in range(n_cmps)
      ))
      if n_cmps > 1 else
      IndependentComponentsDistribution(tuple(
          NormalDistribution(*dist_init[state_i, 0, feat_i, :])
          for feat_i in range(n_features)
      ))
      for state_i in range(n_states)
  )
  return dists

In [ ]:
# Generate progressive HMM model
def left_right_GMMHMM(seed_sample, x_dim, n_states=10, n_modals=9, random=0):
  rng = np.random.RandomState(random)
  if random!=0:
    init_prob = lambda: rng.rand(1)[0]
    
  model = HiddenMarkovModel()
  states = [State( state_model, name=f"H{i}" ) for i, state_model in enumerate(Generate_DiagGMM(seed_sample, x_dim, n_states, n_modals))]
  model.add_states(states)
  model.add_transition(model.start, states[0], 1)
  for i in range(n_states-1):
    model.add_transition(states[i], states[i], 0.5)
    model.add_transition(states[i], states[i+1], 0.5)
  model.add_transition(states[n_states-1], states[n_states-1], 0.5)
  model.add_transition(states[n_states-1], model.end, 0.5)
  model.bake()
  return model

In [ ]:
def train_GMMHMM(dataset, input_dim, n_hidden_state, n_gauss_modal):
    GMMHMM_Models = dict()

    for label in dataset.keys():
        print(f"Training model detect {label}")
        model = left_right_GMMHMM(dataset[label][0], input_dim, n_hidden_state, n_gauss_modal)
        model.fit(dataset[label], verbose=True, multiple_check_input=False)  # get optimal parameters
        GMMHMM_Models[label] = model

    return GMMHMM_Models

## Hàm test mô hình

In [ ]:
# test model
def evaluateModel(testDataSet, model):
    if (len(testDataSet) == 0):
      return
    score_cnt = 0
    total = 0
    for label in testDataSet.keys():
        features = testDataSet[label]
        total += len(features)
        for sample in features:
            scoreList = {}
            for model_label in model.keys():
                score = model[model_label].log_probability(sample)
                scoreList[model_label] = score
            predict = max(scoreList, key=scoreList.get)
            if predict == label:
                score_cnt += 1
        print(f"{label}: ({score_cnt}/{total})")

    print("Final recognition rate is %.2f"%(100.0*score_cnt/total), "%")

## Đọc và tiền xử lý data

In [ ]:
# prepare data for training
master_path = 'fsdd'

trainDir = master_path + '/train_audio/'
trainDataSet_fsdd = buildDataSet(trainDir)
print("Finish prepare the training data")

# prepare data for testing
testDir = master_path + '/test_audio/'
testDataSet_fsdd = buildDataSet(testDir)
print("Finish prepare the test data")

Finish prepare the training data
Finish prepare the test data


In [ ]:
# prepare data for training
master_path = 'wolfram'

trainDir = master_path + '/train_audio/'
trainDataSet_wolfram = buildDataSet(trainDir)
print("Finish prepare the training data")

# prepare data for testing
testDir = master_path + '/test_audio/'
testDataSet_wolfram = buildDataSet(testDir)
print("Finish prepare the test data")

Finish prepare the training data
Finish prepare the test data


In [ ]:
trainDataSet = trainDataSet_fsdd
if wolfram_split > 0:
  for label in trainDataSet_fsdd.keys():
    trainDataSet[label] = trainDataSet_fsdd[label] + trainDataSet_wolfram[label]

## **Mô hình 1**
- 10 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,10,1)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 8411460.682033462	Time (s): 0.5373
[2] Improvement: 22714.003356229863	Time (s): 0.5228
[3] Improvement: 5576.305293732672	Time (s): 0.5315
[4] Improvement: 3224.5079642653	Time (s): 0.5171
[5] Improvement: 1477.4003522814019	Time (s): 0.5183
[6] Improvement: 504.56895708991215	Time (s): 0.5241
[7] Improvement: 291.34769975265954	Time (s): 0.5296
[8] Improvement: 187.50859286484774	Time (s): 0.5174
[9] Improvement: 115.73405086935963	Time (s): 0.531
[10] Improvement: 74.6620602319017	Time (s): 0.521
[11] Improvement: 52.399720237008296	Time (s): 0.5209
[12] Improvement: 50.20381709409412	Time (s): 0.5106
[13] Improvement: 209.5119183093775	Time (s): 0.5204
[14] Improvement: 420.4496254690457	Time (s): 0.5106
[15] Improvement: 292.48109757644124	Time (s): 0.54
[16] Improvement: 163.1507124969503	Time (s): 0.519
[17] Improvement: 117.31236179743428	Time (s): 0.521
[18] Improvement: 91.16633540438488	Time (s): 0.516
[19] Improvement: 92.16243186779

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (218/1614)
3: (632/3193)
2: (1306/4755)
7: (1826/6347)
6: (2179/7971)
8: (2561/9544)
0: (2924/11144)
9: (3428/12761)
5: (4124/14355)
4: (5027/15999)
Final recognition rate is 31.42 %
Evaluate on rest of FSDD set
0: (15/113)
3: (46/243)
9: (55/358)
7: (120/503)
4: (218/618)
5: (284/739)
1: (308/859)
2: (318/973)
6: (335/1081)
8: (407/1200)
Final recognition rate is 33.92 %
Evaluate on wolfram test set
4: (476/913)
1: (604/1849)
6: (775/2786)
8: (951/3746)
0: (1193/4709)
7: (1465/5649)
5: (1872/6591)
2: (2328/7588)
3: (2586/8535)
9: (2903/9467)
Final recognition rate is 30.66 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_10_1[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()

## **Mô hình 2**
- 5 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,5,1)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 19909839.62194393	Time (s): 0.3033
[2] Improvement: 13331.184939627652	Time (s): 0.316
[3] Improvement: 5878.683642033371	Time (s): 0.2939
[4] Improvement: 3162.1008194556925	Time (s): 0.2879
[5] Improvement: 2061.01353536488	Time (s): 0.2994
[6] Improvement: 1567.6995354419341	Time (s): 0.289
[7] Improvement: 1240.743820054573	Time (s): 0.3079
[8] Improvement: 920.892336395802	Time (s): 0.2837
[9] Improvement: 598.8725887706969	Time (s): 0.2892
[10] Improvement: 358.24564699921757	Time (s): 0.2973
[11] Improvement: 204.98151278612204	Time (s): 0.3001
[12] Improvement: 112.64370630600024	Time (s): 0.3104
[13] Improvement: 72.56031027773861	Time (s): 0.296
[14] Improvement: 47.986549460561946	Time (s): 0.3074
[15] Improvement: 28.28244382410776	Time (s): 0.3035
[16] Improvement: 14.29700130072888	Time (s): 0.2987
[17] Improvement: 7.061367829097435	Time (s): 0.2953
[18] Improvement: 3.567150645540096	Time (s): 0.2878
[19] Improvement: 1.825509392

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (360/1614)
3: (471/3193)
2: (1067/4755)
7: (1407/6347)
6: (1751/7971)
8: (2564/9544)
0: (2896/11144)
9: (3160/12761)
5: (3807/14355)
4: (4123/15999)
Final recognition rate is 25.77 %
Evaluate on rest of FSDD set
0: (38/113)
3: (52/243)
9: (72/358)
7: (91/503)
4: (110/618)
5: (177/739)
1: (202/859)
2: (230/973)
6: (244/1081)
8: (312/1200)
Final recognition rate is 26.00 %
Evaluate on wolfram test set
4: (171/913)
1: (363/1849)
6: (573/2786)
8: (1037/3746)
0: (1227/4709)
7: (1444/5649)
5: (1803/6591)
2: (2198/7588)
3: (2267/8535)
9: (2429/9467)
Final recognition rate is 25.66 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_5_1[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()

## **Mô hình 3**
- 3 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,3,1)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 1368083.5862755873	Time (s): 0.2197
[2] Improvement: 25899.19376957661	Time (s): 0.2213
[3] Improvement: 7759.981667786604	Time (s): 0.2427
[4] Improvement: 1822.2883719217498	Time (s): 0.2322
[5] Improvement: 705.2553847130621	Time (s): 0.2196
[6] Improvement: 294.5363751719706	Time (s): 0.2196
[7] Improvement: 158.2376565716695	Time (s): 0.2092
[8] Improvement: 101.32010715838987	Time (s): 0.2011
[9] Improvement: 66.35882048297208	Time (s): 0.224
[10] Improvement: 46.228898949339055	Time (s): 0.2006
[11] Improvement: 29.6460468681762	Time (s): 0.2144
[12] Improvement: 16.696843217941932	Time (s): 0.2145
[13] Improvement: 10.080353707657196	Time (s): 0.2033
[14] Improvement: 6.7104097184492275	Time (s): 0.221
[15] Improvement: 4.668070851126686	Time (s): 0.2103
[16] Improvement: 3.2239620878826827	Time (s): 0.2031
[17] Improvement: 2.193277183221653	Time (s): 0.2025
[18] Improvement: 1.5036559710279107	Time (s): 0.2232
[19] Improvement: 1.04745

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (273/1614)
3: (514/3193)
2: (622/4755)
7: (919/6347)
6: (1204/7971)
8: (1789/9544)
0: (2175/11144)
9: (2434/12761)
5: (2984/14355)
4: (3287/15999)
Final recognition rate is 20.55 %
Evaluate on rest of FSDD set
0: (52/113)
3: (80/243)
9: (100/358)
7: (113/503)
4: (131/618)
5: (180/739)
1: (215/859)
2: (220/973)
6: (228/1081)
8: (273/1200)
Final recognition rate is 22.75 %
Evaluate on wolfram test set
4: (177/913)
1: (302/1849)
6: (465/2786)
8: (818/3746)
0: (997/4709)
7: (1176/5649)
5: (1500/6591)
2: (1573/7588)
3: (1711/8535)
9: (1872/9467)
Final recognition rate is 19.77 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_3_1[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()